## Transfer Learning

### import libraries

In [6]:
import numpy as np
import matplotlib.pyplot as plt
from glob import glob

import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten, Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet import preprocess_input
from tensorflow.keras.backend import clear_session

### VGG16 model

In [7]:
IMAGE_SIZE = [224,224]
vgg = VGG16(input_shape = IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [8]:
#dont train existing weight
for layer in vgg.layers:
    layer.trainable = False

In [9]:
folder = glob('Data/*')
#we need to add more data
x = Flatten()(vgg.output)
prediction = Dense(len(folder), activation = tf.nn.softmax)(x)
model = Model(inputs = vgg.input, outputs = prediction)

In [10]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

In [11]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics = ['accuracy']
)

In [12]:
#import image from datasets using ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  #featurewise_center = False, #set input mean 0 over the data
                                  #samplewise_center = False, #set each sample mean 0
                                  #featurewise_std_normalization=False, #divide inputs by std of the datasets
                                  zca_whitening = False,
                                  rotation_range=10, #randomly rotate images in the range 0-100
                                  width_shift_range = 0.2, #randomly shift image vertically. fraction of total height.
                                  vertical_flip = False
                                  )


training_set = train_datagen.flow_from_directory('Data',
                                                target_size=(224,224),
                                                batch_size=16,
                                                class_mode='categorical')



Found 1420 images belonging to 4 classes.


In [13]:
#train model
r = model.fit_generator(
  training_set,
  epochs=3,
  steps_per_epoch=27,
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/3
27/27 [==============================] - 663s 25s/step - loss: 1.5574 - accuracy: 0.4236
Epoch 2/3
27/27 [==============================] - 655s 24s/step - loss: 0.7890 - accuracy: 0.6752
Epoch 3/3
27/27 [==============================] - 654s 24s/step - loss: 0.8223 - accuracy: 0.6706


In [14]:
model.save('model.h5')

In [15]:
def predict(image_path):
    model = None
    model = load_model('model.h5')
    img = image.load_img(image_path, target_size=(224,224))
    #processing the image
    x = image.img_to_array(img)
    x = x/255  #scaling
    x = np.expand_dims(x, axis=0)
    
    x = preprocess_input(x)
    prediction = model.predict(x)
    clear_session()
    prediction = np.argmax(prediction, axis=1)
    return prediction

In [19]:
predict('11.jpg')

array([1], dtype=int64)